In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import uproot
from pathlib import Path
import awkward as ak
import seaborn as sn
import atlas_mpl_style as ampl
ampl.use_atlas_style()  

In [7]:
# import custom functions from src folder
module_path = str(Path.cwd().parents[0] / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

from imcal import *

In [8]:
N_DIMS = [4]
M_MIN = [8, 10, 12]

BH_labels = [""]*(len(N_DIMS)*len(M_MIN))
i = 0
for n in N_DIMS:
    for M in M_MIN:
        BH_labels[i] = f"BH_n{n}_M{M}"
        i=i+1
BH_data_paths = [f"/disk/atlas3/data_MC/delphes/{label}_10000events.root:Delphes" for label in BH_labels]

sph_data_paths = ["/disk/atlas3/data_MC/delphes/PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_10000events.root:Delphes"]
sph_labels = ["SPH_9TeV"]

##Defines the number of high pT objects, used to perform cut
min_pt = 70
max_eta = 2.4 

n_BH_labels = len(BH_data_paths)
n_sph_labels = len(sph_data_paths)

N_events = 1000

labels = BH_labels + sph_labels
print(labels)

['BH_n4_M8', 'BH_n4_M10', 'BH_n4_M12', 'SPH_9TeV']


In [9]:
def cut_pt_eta(array, pt_min, eta_max):
    array = ak.pad_none(array, 1, axis=-1)
    array = array[array.PT > pt_min]
    array = array[abs(array.Eta) < eta_max]
    n = np.array([len(event) for event in array.PT])
    return array, n

def cut_pt_eta_met(array, pt_min, eta_max):
    array = ak.pad_none(array, 1, axis=-1)
    array = array[array.MET > pt_min]
    array = array[abs(array.Eta) < eta_max]
    return array

def calculate_ST(jets, muons, electrons, photons, met):
    ST = np.zeros(len(jets))
    jet_sum = np.sum(jets.PT, axis=-1)/1000
    muon_sum = np.sum(muons.PT, axis=-1)/1000
    electron_sum = np.sum(electrons.PT, axis=-1)/1000
    photon_sum = np.sum(photons.PT, axis=-1)/1000
    met_sum = np.sum(met.MET, axis=-1)/1000
    ST = jet_sum + muon_sum + electron_sum + photon_sum + met_sum
    return ST


These are some examples of how to look at root files using uproot and awkward arrays. More info in this tutorial:
https://hub.gke2.mybinder.org/user/jpivarski-2020--ep2020-tutorial-7h7oraqf/lab/tree/tutorial.ipynb


In [10]:
#Open file in with-function will close it when you exit

def get_arrays(data_paths):
    clusters = [load_data(path, "Tower", 
                            ["Tower.ET", "Tower.Eta", "Tower.Phi", "Tower.Eem", "Tower.Ehad", "Tower.E"], N_events)
                            for path in data_paths]

    jets = [load_data(path, "Jet", 
                            ["Jet.PT", "Jet.Eta", "Jet.Phi"], N_events)
                            for path in data_paths]
                
    met = [load_data(path, "MissingET", 
                            ["MissingET.MET", "MissingET.Eta", "MissingET.Phi"], N_events)
                            for path in data_paths]

    electrons = [load_data(path, "Electron", 
                            ["Electron.PT", "Electron.Eta", "Electron.Phi", "Electron.Charge"], N_events)
                            for path in data_paths]

    muons = [load_data(path, "Muon", 
                            ["Muon.PT", "Muon.Eta", "Muon.Phi", "Muon.Charge"], N_events)
                            for path in data_paths]

    photons = [load_data(path, "Photon", 
                            ["Photon.PT", "Photon.Eta", "Photon.Phi"], N_events)
                            for path in data_paths]

    return clusters, jets, met, electrons, muons, photons

BH_clusters, BH_jets, BH_met, BH_electrons, BH_muons, BH_photons = get_arrays(BH_data_paths)
sph_clusters, sph_jets, sph_met, sph_electrons, sph_muons, sph_photons = get_arrays(sph_data_paths)

print([len(BH_clusters[i][0]) for i in range(len(BH_clusters))])
print([len(BH_clusters[i]) for i in range(len(BH_clusters))])

[312, 269, 183]
[1000, 1000, 1000]


# Jet data

In [11]:
#Extracting data for plotting from jets
def jet_data (jets):
    jets = [item[item.PT > min_pt] for item in jets]
    jets = [item[abs(item.Eta) < max_eta] for item in jets]
    jets = [ak.pad_none(item, 1, axis=-1) for item in jets]
    n_jets = [np.array([len(event) for event in item.PT]) for item in jets]
    jet1_PT = [ak.to_list(item.PT[:,0]) for item in jets]
    jet1_eta = [ak.to_list(item.Eta[:,0]) for item in jets]
    return jets, n_jets, jet1_PT, jet1_eta

BH_jets, n_BH_jets, BH_jet1_PT, BH_jet1_eta = jet_data(BH_jets)
sph_jets, n_sph_jets, sph_jet1_PT, sph_jet1_eta = jet_data(sph_jets)

# Multiplicity

In [12]:
#Extracting data for plotting from particle data
def multiplicity(photons, electrons, muons):
    photons = [item[item.PT > min_pt] for item in photons]
    photons = [item[abs(item.Eta) < max_eta] for item in photons]
    n_photons = [[len(event) for event in item.PT] for item in photons]
    electrons = [item[item.PT > min_pt] for item in electrons]
    electrons = [item[abs(item.Eta) < max_eta] for item in electrons]
    n_electrons = [[len(event) for event in item.PT] for item in electrons]
    #separate muons by charge
    muons = [item[item.PT > min_pt] for item in muons]
    muons = [item[abs(item.Eta) < max_eta] for item in muons]
    muons_neg = [item[item.Charge < 0] for item in muons]
    muons_pos = [item[item.Charge > 0] for item in muons]
    n_muons_neg = [np.array([len(event) for event in item.PT]) for item in muons_neg]
    n_muons_pos = [np.array([len(event) for event in item.PT]) for item in muons_pos]
    n_muons = [np.array([len(event) for event in item.PT]) for item in muons]
    return n_photons, n_electrons, n_muons_neg, n_muons_pos, n_muons 

n_BH_photons, n_BH_electrons, n_BH_muons_neg, n_BH_muons_pos, n_BH_muons = multiplicity(BH_photons, BH_electrons, BH_muons)
n_sph_photons, n_sph_electrons, n_sph_muons_neg, n_sph_muons_pos, n_sph_muons = multiplicity(sph_photons, sph_electrons, sph_muons)


In [13]:
#ST = scalar sum of all jets, leptons, photons and MET
def calculate_ST(n_labels, jets, muons, electrons, photons, met):
    ST = np.zeros((n_labels, N_events))
    for i in range(n_labels):
        jet_sum = np.sum(jets[i].PT, axis=-1)/1000
        muon_sum = np.sum(muons[i].PT, axis=-1)/1000
        electron_sum = np.sum(electrons[i].PT, axis=-1)/1000
        photon_sum = np.sum(photons[i].PT, axis=-1)/1000
        met_sum = np.sum(met[i].MET, axis=-1)/1000
        k = 20
        ST[i] = jet_sum + muon_sum + electron_sum + photon_sum + met_sum
    return ST
BH_ST = calculate_ST(n_BH_labels, BH_jets, BH_muons, BH_electrons, BH_photons, BH_met)
sph_ST = calculate_ST(n_sph_labels, sph_jets, sph_muons, sph_electrons, sph_photons, sph_met)

In [14]:
#Cuts
BH_N = np.array(n_BH_jets) + np.array(n_BH_electrons) + np.array(n_BH_muons) + np.array(n_BH_photons)
sph_N = np.array(n_sph_jets) + np.array(n_sph_electrons) + np.array(n_sph_muons) + np.array(n_sph_photons)

#Dictionary 
df_dict = {}
for i, label in enumerate(BH_labels):
    df_dict[label] = pd.DataFrame({"N":BH_N[i], "ST":BH_ST[i]})

for i, label in enumerate(sph_labels):
    df_dict[label] = pd.DataFrame({"N":sph_N[i], "ST":sph_ST[i]})


In [15]:
def efficiency(dictionary, N_cut, ST_cut, labels, file):
    print(f"Efficiency for N >= {N_cut} and ST >= {ST_cut}:")
    file.write(f"Efficiency for N >= {N_cut} and ST >= {ST_cut}:\n")
    for label in labels:
        df = dictionary[label]
        N_before = len(df)
        df = df[df["N"] >= N_cut]
        df = df[df["ST"] >= ST_cut] 
        N_after = len(df)
        print(f"{label}: {np.round(N_after/N_before, 2)}")
        file.write(f"{label}: {np.round(N_after/N_before, 2)}\n")

N_cuts = [5]
ST_cuts = [6, 7]

file = open("../results/Efficiencies_paper.txt", "w")

for N_cut in N_cuts:
    for ST_cut in ST_cuts:
        efficiency(df_dict, N_cut, ST_cut, labels, file)
file.close()

Efficiency for N >= 5 and ST >= 6:
BH_n4_M8: 0.41
BH_n4_M10: 0.63
BH_n4_M12: 0.7
SPH_9TeV: 0.57
Efficiency for N >= 5 and ST >= 7:
BH_n4_M8: 0.18
BH_n4_M10: 0.5
BH_n4_M12: 0.65
SPH_9TeV: 0.19
